In [1]:
import pandas as pd
import glob
import os
import keras

In [2]:
    #save the models to model_folder
    model_folder='./models'
    #output folder of trajactory got from the simulation
    output_folder='./output'
    #create energy_folder to save the prediction results
    energy_folder='./energy'

# Select scenario for weather condition

In [3]:
#weather option: rainy, sunny, windy, and snowy
weather = 'sunny'

# Load microscopic energy eatimation model for different vehicle classes


In [4]:
model_diesel =  keras.models.load_model(f'{model_folder}/diesel.model')
model_hybrid =  keras.models.load_model(f'{model_folder}/hybrid.model')
model_electric =  keras.models.load_model(f'{model_folder}/electric.model')

# Predict energy consumption of each trip 

In [5]:
path = f'{output_folder}'                    
all_files = glob.glob(os.path.join(path, "Trajectory*.csv")) 
colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass']
for f in all_files:
    trip=pd.read_csv(f,names=colnames, header=None)
    #trip=pd.read_csv(f)
    trip = trip[trip.speed != 'speed']
    trip = trip[trip.acceleration != 'acceleration']
    trip['speed']=trip['speed'].astype(float)*0.036 #km/h
    #find the vehicle class of this trip based on the column "actorConfig_ref"
    vtype = ['Gillig_LF_2014','Gillig_HF_2006','Gillig_HF_2002','Gillig_HF_1998','Gillig_Hybrids_2014',
             'Gillig_Hybrids_2009','Gillig_LF_2009','BYD_BEV']
    diesel = ['Gillig_LF_2014','Gillig_HF_2006','Gillig_HF_2002','Gillig_HF_1998','Gillig_LF_2009']
    hybrid = ['Gillig_Hybrids_2014', 'Gillig_Hybrids_2009']
    electric = ['BYD_BEV']
    # print(trip['actorConfig_fuel'].iloc[2])
    if trip['actorConfig_fuel'].iloc[2] in 'Gasoline':
        Vclass = 'diesel'
    elif trip['actorConfig_fuel'].iloc[2] in 'Hybrid':
        Vclass = 'hybrid'
    elif trip['actorConfig_fuel'].iloc[2] in "Electric":
        Vclass = 'electric'
    if Vclass == 'electric':
        trip['acceleration']=trip['acceleration'].astype(float)*0.001*3.28084
        #ft/s2
    else:
        trip['acceleration']=trip['acceleration'].astype(float)*0.001 
        #m/s2
    if weather == 'rainy':
        trip['temperature']= 60 
        trip['humidity']= 0.9 #range[0,1]
    if weather == 'sunny':
        trip['temperature']= 70
        trip['humidity']= 0.5
    if weather == 'windy':
        trip['temperature']= 60
        trip['humidity']= 0.6
    if weather == 'snowy':
        trip['temperature']= 30
        trip['humidity']= 0.4
    trip['roadGrade']= 0
    input2esti=trip[['speed','acceleration','temperature','humidity','roadGrade']]
    #prdiction 
    if Vclass == 'electric':
        pre = model_electric.predict(input2esti) 
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1)
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/33.7  #gal
        path_new = os.path.join(energy_folder, Vclass)
        try:
            os.mkdir(path_new)
        except:
            pass
        with open(f'{path_new}'+ f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)    
    elif Vclass == 'hybrid':
        pre = model_hybrid.predict(input2esti)
        pre[pre<0]=0
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1) 
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/3600  #gal
        path_new = os.path.join(energy_folder, Vclass)
        try:
            os.mkdir(path_new)
        except:
            pass        
        with open(f'{path_new}' + f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    elif Vclass == 'diesel':
        pre = model_diesel.predict(input2esti)
        pre[pre<0]=0
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energyrate_'+Vclass])], axis=1)    
        tripf['energy_gal'] = tripf['energyrate_'+Vclass]/3600  #gal
        path_new = os.path.join(energy_folder, Vclass)
        try:
            os.mkdir(path_new)
        except:
            pass
        #write the predicted results 
        with open(f'{path_new}' + f[len(output_folder):] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    
    

# Estimate energy consumption of each route

In [6]:
# Route = ['Route1','Route2A','Route3','Route4','Route7','Route8','Route9','Route10C','Route10A','Route10G','Route16','Route21','Route33','Route34','Route13','Route15A','Route21A','Route28','RouteDTS']
# route_summary = pd.DataFrame()
# b=0
# excep = []
# route_energy = pd.DataFrame()
# types = ['diesel', 'hybrid', 'electric']
# types = [ 'electric']
# for typ in types:
#     path = os.path.join(f'{energy_folder}', typ)
#     for r in Route: 
#         allfiles = glob.glob(os.path.join(path, "*"+r+"*.csv"))
#         # print(os.path.join(path, "*"+r+"*.csv"))
#         if len(allfiles) > 0:
#             colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass','temperature','humidity','roadGrade','energyrate','energy_gal']
#             c=0
#             for f in allfiles:
#                 df=pd.read_csv(f,names=colnames, header=None)
#                 df['speed']=pd.to_numeric(df['speed'],errors='coerce')
#                 df['energy_gal']=pd.to_numeric(df['energy_gal'],errors='coerce')    
#                 df['speed_mile_s']=df['speed']*0.000172603  #mile/s 
#                 energy=df['speed_mile_s'].sum()/df['energy_gal'].sum() #mi/gal
#                 sum_gal = df['energy_gal'].sum()
#                 route_energy.loc[c,'energy_mpg']=energy
#                 route_energy.loc[c,'sum_energy_gal']=sum_gal
#                 with open(f'{energy_folder}'+'_'+ r + 'energy.csv', 'w', newline='') as wFile:
#                     route_energy.to_csv(wFile, index = False)
#                 # c+=1
#             try:
#                 route_energy['trip_id']=[f[-24:-15] for f in allfiles]
#             except :
#                 excep.append([f[-24:-8] for f in allfiles])
#                 print([f[-24:-15] for f in allfiles])
#                 pass
#             route_summary.loc[b,'sum_energy_gal']=route_energy['sum_energy_gal'].sum()
#             route_summary.loc[b,'route']=r
#             b+=1
# print(route_summary)

Empty DataFrame
Columns: []
Index: []


In [31]:
import datetime

def conv_time(ms):
    s = int(ms)/1000
    time = str(datetime.timedelta(seconds=s))
    return time

In [38]:
route_summary = pd.DataFrame()
b=0
excep = []
route_energy = pd.DataFrame()
types = ['diesel', 'hybrid', 'electric']
types = [ 'hybrid']
c=0
for typ in types:
    path = os.path.join(f'{energy_folder}', typ)
    allfiles = glob.glob(os.path.join(path, "Trajectory_"+"*.csv"))
    for f in allfiles:
        if len(allfiles) > 0:
            colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass','temperature','humidity','roadGrade','energyrate','energy_gal']
            df=pd.read_csv(f,names=colnames, header=None)
            df['speed']=pd.to_numeric(df['speed'],errors='coerce')
            df['energy_gal']=pd.to_numeric(df['energy_gal'],errors='coerce')    
            df['speed_mile_s']=df['speed']*0.000172603  #mile/s 
            energy=df['speed_mile_s'].sum()/df['energy_gal'].sum() #mi/gal
            sum_gal = df['energy_gal'].sum()
            route_energy.loc[b, 'bus_id']= df.vehicle_ref.iloc[2].split('_')[0]
            route_energy.loc[b, 'trip_id']= df.vehicle_ref.iloc[2].split('_')[1]
            route_energy.loc[b, 'start_time']= df.vehicle_ref.iloc[2].split('_')[2]
            route_energy.loc[b, 'actual_start_time']=  conv_time(df.time_ms.iloc[2])
            route_energy.loc[b, 'end_time']=  conv_time(df.time_ms.iloc[-1])
            route_energy.loc[b,'energy_mpg']=energy
            route_energy.loc[b,'sum_energy_gal']=sum_gal
            route_energy.loc[b,'sum_energy_kWh']=sum_gal*33.7
            # c+=1
            k = [chars for chars in f]
            # print(''.join(k[-24:-15]))
            # k=[]
            b+=1
print(route_energy.head())

   bus_id trip_id start_time actual_start_time  end_time  energy_mpg  \
0  506-11  206314   14:15:00          14:52:09  16:46:41    3.572161   
1   506-9  205918   12:26:00          12:26:00  13:13:49    4.059564   
2   501-3  206344   08:00:00           9:31:07  10:38:27    3.960442   
3   501-2  206301   07:00:00           8:07:52   9:31:05    3.964973   
4  506-16  205178   17:40:00          20:39:06  21:09:06    4.023989   

   sum_energy_gal  sum_energy_kWh  
0        3.939468      132.760077  
1        1.743255       58.747679  
2        3.539384      119.277231  
3        3.551902      119.699087  
4        1.736712       58.527181  


In [39]:
with open(f'{energy_folder}\{types[0]}_energy_trip.csv', 'w', newline='') as oFile:
    route_energy.to_csv(oFile, index = False)
route_energy

,bus_id,trip_id,start_time,actual_start_time,end_time,energy_mpg,sum_energy_gal,sum_energy_kWh
0,506-11,206314,14:15:00,14:52:09,16:46:41,3.572161,3.939468,132.760077
1,506-9,205918,12:26:00,12:26:00,13:13:49,4.059564,1.743255,58.747679
2,501-3,206344,08:00:00,9:31:07,10:38:27,3.960442,3.539384,119.277231
3,501-2,206301,07:00:00,8:07:52,9:31:05,3.964973,3.551902,119.699087
4,506-16,205178,17:40:00,20:39:06,21:09:06,4.023989,1.736712,58.527181
5,501-0,206312,04:51:00,4:51:40,6:49:26,3.724151,4.834683,162.928802
6,501-10,206764,16:10:00,19:45:21,20:18:55,3.547132,1.964664,66.209177
7,504-12,205161,16:10:00,17:33:01,19:48:11,2.671763,3.158034,106.425755
8,504-7,205073,12:20:00,13:49:13,14:10:49,4.320556,1.148245,38.695848
9,504-5,206343,09:30:00,11:31:21,12:57:31,4.104047,4.433058,149.394045


In [85]:
route_energy

,energy_mpg,sum_energy_gal,trip_id
0,9.769187,0.215163,206082020
1,9.708715,0.170004,206082020
2,10.676766,0.878602,206082020
3,16.141023,0.263973,206082020
4,9.418754,0.188057,206082020
...,...,...,...
87,9.567784,0.173410,206082020
88,10.681661,0.151937,206082020
89,11.864836,0.204026,206082020
90,9.401175,0.223579,206082020
